In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
from datetime import datetime
import time
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.metrics import accuracy_score,f1_score,recall_score,precision_score,confusion_matrix,ConfusionMatrixDisplay,roc_curve,roc_auc_score,precision_recall_curve
from sklearn.ensemble import RandomForestClassifier , StackingClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression,Lasso
from sklearn.preprocessing import Binarizer
from sklearn.model_selection import cross_val_score,GridSearchCV

# 한글 깨짐 방지
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'malgun Gothic'

In [2]:
train = pd.read_csv('../data/train_Winsorization_시총_상장.csv', encoding='euc-kr')
test = pd.read_csv('../data/test_시총_상장.csv', encoding='euc-kr')

In [3]:
X_train_sum = train[['부채비율', '총자본회전률', '매출액대비잉여현금흐름', 'PBR',
       '총자산대비영업현금흐름', '자기자본증가율', '총자본투자효율', '총자본순이익률',
       '매출액영업이익률']]
y_train = train[['t-1감사의견코드']]
X_test_sum = test[['부채비율', '총자본회전률', '매출액대비잉여현금흐름', 'PBR',
       '총자산대비영업현금흐름', '자기자본증가율', '총자본투자효율', '총자본순이익률',
       '매출액영업이익률']]
y_test = test[['t-1감사의견코드']]

In [4]:
X_train = train.drop('t-1감사의견코드',axis=1)
y_train = train[['t-1감사의견코드']]

X_test = test.drop('t-1감사의견코드',axis=1)
y_test = test[['t-1감사의견코드']]

In [5]:
from imblearn.under_sampling import OneSidedSelection

# One Side Selection을 사용하여 언더샘플링된 데이터 생성
oss = OneSidedSelection()
X_test_sum, y_test = oss.fit_resample(X_test_sum, y_test)
X_train_sum, y_train = oss.fit_resample(X_train_sum, y_train)

In [6]:
scaler = StandardScaler()
X_train_sc = scaler.fit_transform(X_train_sum)
X_test_sc = scaler.transform(X_test_sum)

In [7]:
X_train_sc=pd.DataFrame(X_train_sc, columns=X_train_sum.columns)
X_test_sc=pd.DataFrame(X_test_sc, columns=X_test_sum.columns)

In [8]:
X_test_sc=pd.concat([X_test_sc,y_test],axis=1)
X_train_sc=pd.concat([X_train_sc,y_train],axis=1)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.utils import resample

# X_train_sc, y_train, X_test_sc, y_test 데이터가 이미 준비되어 있다고 가정합니다.
# 여기서는 임의의 데이터를 생성하여 예시로 사용합니다.
np.random.seed(0)
X_train_sc = pd.DataFrame(np.random.randn(100, 10))
y_train = pd.Series(np.random.choice([0, 1], size=100, replace=True))
X_test_sc = pd.DataFrame(np.random.randn(30, 10))
y_test = pd.Series(np.random.choice([0, 1], size=30, replace=True))

# 합친 데이터를 생성합니다.
train_data = pd.concat([X_train_sc, y_train], axis=1)

# 소수 클래스의 개수를 확인합니다.
minority_class = train_data[train_data.iloc[:, -1] == 1]
n_minority_class = len(minority_class)

# 다수 클래스의 개수에 맞게 랜덤 언더샘플링을 수행합니다.
majority_class = train_data[train_data.iloc[:, -1] == 0].sample(n=n_minority_class, random_state=0)

# 언더샘플링한 데이터를 합칩니다.
under_sampled_data = pd.concat([minority_class, majority_class])

# 언더샘플링한 데이터를 다시 X_train, y_train으로 분리합니다.
X_train = under_sampled_data.iloc[:, :-1]
y_train = under_sampled_data.iloc[:, -1]

print("언더샘플링 전:")
print(y_train.value_counts())

# 언더샘플링 후의 데이터 개수를 출력합니다.
print("언더샘플링 후:")
print(y_train.value_counts())
